In [2]:
import pandas as pd
import geopandas as gp
import numpy as np
import os
import requests
import urllib.parse

In [3]:
shp_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Shareholder Proposal/Robin_and_BJ_tour.shp"
out_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Shareholder Proposal/Robin_and_BJ_tour.csv"

In [16]:
shp = gp.read_file(shp_path)
shp.to_crs('4326', inplace = True)
shp['Latitude'] = shp.geometry.y
shp['Longitude'] = shp.geometry.x
shp['SAMPLED'] = ''
shp[['Field_Note','Latitude', 'Longitude']].to_csv(out_path)

In [17]:
shp.to_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Shareholder Proposal/Robin_and_BJ_tour.geojson", driver = 'GeoJSON')

GNSS Processing
- takes GNSS points from arcgis field collector app and joins to screening results

In [8]:
## GNSS Processing
url_base = "https://gisdata.seattle.gov/server/rest/services/SPU/SPU_DWW/MapServer/6"
response = requests.get(url=url_base)
params = {
    'geometryType': 'esriGeometryPoint',
    'inSR': '4326',
    'distance': '10000', 
    'units': 'esriSRUnit_Meter', 
    'returnGeometry': 'true'
    }
url_final = url_base + urllib.parse.urlencode(params)
response = requests.get(url=url_final)
dir(response)
data = response.json
#gdf_temp = gp.read_file(data)

In [10]:
response

/home/nweiss/miniconda3/envs/gis/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [ ]:

# bring in GNSS points
gnss_pts = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Sampling/GNSS_20231105.shp"
gnss_pts_gdf = gp.read_file(gnss_pts)

# bring in screening results
results = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/Screening_Results20231105.csv"
results_df = pd.read_csv(results)


In [37]:
dup_row_s41 = gnss_pts_gdf[gnss_pts_gdf['Field_Note']=='S4W-1 and S4S-1'].copy()
dup_row_s42 = gnss_pts_gdf[gnss_pts_gdf['Field_Note']=='S4W-2 and S4W-2'].copy() 

In [38]:
dup_row_s41['Field_Note'] = 'S4W-1'
gnss_pts_gdf['Field_Note'] = np.where(gnss_pts_gdf['Field_Note']== 'S4W-1 and S4S-1','S4S-1',gnss_pts_gdf['Field_Note'])

In [39]:
dup_row_s42['Field_Note'] = 'S4W-2'
gnss_pts_gdf['Field_Note'] = np.where(gnss_pts_gdf['Field_Note']== 'S4W-2 and S4W-2','S4S-2',gnss_pts_gdf['Field_Note'])

In [43]:
gnss_pts_gdf = pd.concat([gnss_pts_gdf,dup_row_s41, dup_row_s42])

In [44]:
gnss_pts_gdf['Field_Note'] = np.where(gnss_pts_gdf['Field_Note']=='Dww-6','DWW-6',gnss_pts_gdf['Field_Note'])
gnss_pts_gdf['Field_Note'] = np.where(gnss_pts_gdf['Field_Note']=='DWw-5c','DWW-5C',gnss_pts_gdf['Field_Note'])
gnss_pts_gdf['Field_Note'] = np.where(gnss_pts_gdf['Field_Note']=='Dww 4','DWW-4',gnss_pts_gdf['Field_Note'])
gnss_pts_gdf['Field_Note'] = np.where(gnss_pts_gdf['Field_Note']=='Dws-5a','DWS-5A',gnss_pts_gdf['Field_Note'])
gnss_pts_gdf['Field_Note'] = np.where(gnss_pts_gdf['Field_Note']=='Dws-4, correct location','DWS-4',gnss_pts_gdf['Field_Note'])


In [46]:
results_merge = results_df.merge(gnss_pts_gdf[['Field_Note','geometry']],how = 'left' ,left_on ='Samp_ID', right_on='Field_Note')

In [48]:
results_gdf = gp.GeoDataFrame(results_merge)

In [53]:
results_gdf.to_crs('4326', inplace = True)
out_path = '/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Sampling/sampling_screening_results_20231105.geojson'
geo = results_gdf.to_file(out_path, driver = 'GeoJSON')

In [11]:
## tutorial of ESRI API
## https://developers.arcgis.com/python/guide/download-data/

from arcgis.gis import GIS
from pathlib import Path
from zipfile import ZipFile

gis = GIS()


In [24]:
public_data_item_id = '5b9edce6791f406fa8fa391e55e07345'

data_item = gis.content.get(public_data_item_id)
print(data_item)

# configure where to save the data, and where the ZIP file is located
data_path = Path('./data')
if not data_path.exists():
    data_path.mkdir()
zip_path = data_path.joinpath('LA_Hub_Datasets.zip')
extract_path = data_path.joinpath('LA_Hub_datasets')
data_item.download(save_path=data_path)

zip_file = ZipFile(zip_path)
zip_file.extractall(path=data_path)

files = [file.name for file in extract_path.glob('*')]
files

<Item title:"DWW Storm Outfalls" type:Feature Layer Collection owner:PublicUtilities_SeattleCityGIS>


['Parks_and_Open_Space.zip', 'Trailheads.csv', 'Trails.geojson']